In [1]:
from pymongo import MongoClient
dbName = "companies"
mongodbURL = f"mongodb://localhost/{dbName}"
client = MongoClient(mongodbURL, connectTimeoutMS=2000, serverSelectionTimeoutMS=2000)
db = client.get_database()

In [2]:
import re
comp= {"name":{"$exists":True}}
cur = db.companies.find(comp)
comp2 = list(cur)

proyection ={"offices":1, "name":1,"category_code":1,"number_of_employees":1,"founded_year":1,"total_money_raised":1,}

comp_result= list(db.companies.find(comp,proyection))



In [3]:
import pandas as pd

df = pd.DataFrame(list(comp_result))
df

,_id,name,category_code,number_of_employees,founded_year,total_money_raised,offices
0,52cdef7c4bab8bd675297d8b,AdventNet,enterprise,600.0,1996.0,$0,"[{'description': 'Headquarters', 'address1': '..."
1,52cdef7c4bab8bd675297d8c,Zoho,software,1600.0,2005.0,$0,"[{'description': 'Headquarters', 'address1': '..."
2,52cdef7c4bab8bd675297d8f,Omnidrive,network_hosting,NaN,2005.0,$800k,"[{'description': '', 'address1': 'Suite 200', ..."
3,52cdef7c4bab8bd675297d8a,Wetpaint,web,47.0,2005.0,$39.8M,"[{'description': '', 'address1': '710 - 2nd Av..."
4,52cdef7c4bab8bd675297d90,Postini,web,NaN,1999.0,$0,"[{'description': None, 'address1': '959 Skyway..."
...,...,...,...,...,...,...,...
18796,52cdef7f4bab8bd67529c6f8,goBookmaker,web,NaN,NaN,$0,[]
18797,52cdef7f4bab8bd67529c6f9,EnteGreat Solutions,software,NaN,NaN,$0,"[{'description': '', 'address1': '', 'address2..."
18798,52cdef7f4bab8bd67529c6fa,EnteGreat,enterprise,NaN,2000.0,$0,"[{'description': '', 'address1': '1900 Interna..."
18799,52cdef7f4bab8bd67529c6f7,AfterLogic,software,NaN,2002.0,$0,"[{'description': 'Livingston', 'address1': 'P...."


In [4]:
offices = df.explode("offices").apply(lambda e: e.offices,axis=1,result_type="expand")
offices

,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,Headquarters,4900 Hopyard Rd.,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
1,Headquarters,4900 Hopyard Rd,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
2,,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,NaN,NaN
3,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253
3,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431
...,...,...,...,...,...,...,...,...,...
18796,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18797,,,,,Birmingham,AL,USA,33.518885,-86.816068
18798,,1900 International Park Drive,Suite 200,35243-4204,Birmingham,AL,USA,33.407786,-86.761302
18799,Livingston,P.O. Box 2096,,07039,Livingston,NJ,USA,40.793024,-74.323554


In [5]:
clean_offices = pd.concat([df[["name","_id","total_money_raised","number_of_employees","category_code","founded_year"]], offices], axis=1)
clean_offices

,name,_id,total_money_raised,number_of_employees,category_code,founded_year,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,AdventNet,52cdef7c4bab8bd675297d8b,$0,600.0,enterprise,1996.0,Headquarters,4900 Hopyard Rd.,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
1,Zoho,52cdef7c4bab8bd675297d8c,$0,1600.0,software,2005.0,Headquarters,4900 Hopyard Rd,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
2,Omnidrive,52cdef7c4bab8bd675297d8f,$800k,NaN,network_hosting,2005.0,,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,NaN,NaN
3,Wetpaint,52cdef7c4bab8bd675297d8a,$39.8M,47.0,web,2005.0,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253
3,Wetpaint,52cdef7c4bab8bd675297d8a,$39.8M,47.0,web,2005.0,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18796,goBookmaker,52cdef7f4bab8bd67529c6f8,$0,NaN,web,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18797,EnteGreat Solutions,52cdef7f4bab8bd67529c6f9,$0,NaN,software,NaN,,,,,Birmingham,AL,USA,33.518885,-86.816068
18798,EnteGreat,52cdef7f4bab8bd67529c6fa,$0,NaN,enterprise,2000.0,,1900 International Park Drive,Suite 200,35243-4204,Birmingham,AL,USA,33.407786,-86.761302
18799,AfterLogic,52cdef7f4bab8bd67529c6f7,$0,NaN,software,2002.0,Livingston,P.O. Box 2096,,07039,Livingston,NJ,USA,40.793024,-74.323554


In [6]:
clean_offices["category_code"].value_counts()

web                 4054
software            3392
mobile              1302
games_video         1200
advertising         1183
other               1155
enterprise           946
consulting           807
ecommerce            777
network_hosting      699
public_relations     619
search               496
hardware             461
biotech              402
cleantech            359
semiconductor        192
security             190
analytics             94
finance               66
news                  64
social                63
legal                 51
music                 50
messaging             47
education             43
photo_video           31
travel                29
health                28
medical               27
manufacturing         22
transportation        16
hospitality           14
sports                14
automotive            11
fashion               10
real_estate           10
design                 6
nanotech               5
nonprofit              4
local                  1


In [7]:
categories = ["web","software","mobile","games_video","advertising","ecommerce","network_hosting","search","music"]
usa2 = clean_offices[(clean_offices["category_code"].isin(categories))]

In [8]:
usa2.dtypes

name                    object
_id                     object
total_money_raised      object
number_of_employees    float64
category_code           object
founded_year           float64
description             object
address1                object
address2                object
zip_code                object
city                    object
state_code              object
country_code            object
latitude               float64
longitude              float64
dtype: object

In [9]:
usa2["total_money_raised"] = usa2["total_money_raised"].replace({'\$':''}, regex = True)
usa2["total_money_raised"] = usa2["total_money_raised"].replace({'\€':''}, regex = True)
usa2["total_money_raised"] = usa2["total_money_raised"].replace({'\£':''}, regex = True)

<ipython-input-9-138dcc907f2e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usa2["total_money_raised"] = usa2["total_money_raised"].replace({'\$':''}, regex = True)
<ipython-input-9-138dcc907f2e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usa2["total_money_raised"] = usa2["total_money_raised"].replace({'\€':''}, regex = True)
<ipython-input-9-138dcc907f2e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

In [10]:
usa2['Last_Digit'] = usa2['total_money_raised'].str.strip().str[-1]
usa2['First_Digit'] = usa2['total_money_raised'].str.strip().str[0]

<ipython-input-10-86baed75a5ce>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usa2['Last_Digit'] = usa2['total_money_raised'].str.strip().str[-1]
<ipython-input-10-86baed75a5ce>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usa2['First_Digit'] = usa2['total_money_raised'].str.strip().str[0]


In [54]:
cat= ['k','M','B']
usa3= usa2[(usa2["Last_Digit"].isin(cat))]

In [12]:
usa3["Last_Digit"].value_counts()

M    3285
k     655
B       8
Name: Last_Digit, dtype: int64

In [13]:
exclude =["1","2","3","4","5","6","7","8","9"]
usa3= usa3[(usa3["First_Digit"].isin(exclude))]

In [14]:
multipliers = {'k':1000, 'M':1000000, 'B':1000000000}

In [15]:
count=0
for i in usa3.total_money_raised:
    if i[-1].isdigit(): # check if no suffix
        print(i)
        usa3["total_money_raised"].iloc[count] = int(string)
        count+=1
    else:
        mult = multipliers[i[-1]] # look up suffix to get multiplier
        usa3["total_money_raised"].iloc[count] = int(float(i[:-1]) * mult)
        count+=1

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [16]:
usa3

,name,_id,total_money_raised,number_of_employees,category_code,founded_year,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,Last_Digit,First_Digit
2,Omnidrive,52cdef7c4bab8bd675297d8f,800000,NaN,network_hosting,2005.0,,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,NaN,NaN,k,8
3,Wetpaint,52cdef7c4bab8bd675297d8a,39800000,47.0,web,2005.0,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253,M,3
3,Wetpaint,52cdef7c4bab8bd675297d8a,39800000,47.0,web,2005.0,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431,M,3
6,Geni,52cdef7c4bab8bd675297d91,16500000,18.0,web,2006.0,Headquarters,9229 W. Sunset Blvd.,,90069,West Hollywood,CA,USA,34.090368,-118.393064,M,1
9,StumbleUpon,52cdef7c4bab8bd675297d95,18500000,NaN,web,2002.0,,,,,San Francisco,CA,USA,37.775196,-122.419204,M,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18733,Tiny Speck,52cdef7f4bab8bd67529c6b6,17200000,45.0,games_video,2009.0,,163 Hastings St W,,BC V6B 1H5,Vancouver,None,CAN,49.282455,-123.109217,M,1
18768,iGuiders,52cdef7f4bab8bd67529c6d9,1170000,5.0,web,2008.0,,23811 Chagrin,Boulevard Suite 244,44122,Beachwood,OH,USA,NaN,NaN,M,1
18779,Getyoo,52cdef7f4bab8bd67529c6e4,1130000,10.0,mobile,2009.0,HQ,"Rue des Tanneurs, 60a",,1000,Brussels,None,BEL,50.839970,4.346472,M,1
18782,OfficialVirtualDJ,52cdef7f4bab8bd67529c6e6,25000000,102.0,music,2009.0,Created in London,11 Eggleston view,,DL3 9SH,London,DC,GBR,NaN,NaN,M,2


In [17]:
def drop_null(table,subset):
    table.dropna(subset=[f"{subset}"],inplace=True)

In [18]:
drop_null(usa3,"longitude")

In [19]:
usa3

,name,_id,total_money_raised,number_of_employees,category_code,founded_year,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,Last_Digit,First_Digit
3,Wetpaint,52cdef7c4bab8bd675297d8a,39800000,47.0,web,2005.0,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253,M,3
3,Wetpaint,52cdef7c4bab8bd675297d8a,39800000,47.0,web,2005.0,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431,M,3
6,Geni,52cdef7c4bab8bd675297d91,16500000,18.0,web,2006.0,Headquarters,9229 W. Sunset Blvd.,,90069,West Hollywood,CA,USA,34.090368,-118.393064,M,1
9,StumbleUpon,52cdef7c4bab8bd675297d95,18500000,NaN,web,2002.0,,,,,San Francisco,CA,USA,37.775196,-122.419204,M,1
10,Gizmoz,52cdef7c4bab8bd675297d96,18100000,NaN,web,2003.0,None,None,None,None,Menlo Park,CA,USA,37.484130,-122.169472,M,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18719,Vigilos,52cdef7f4bab8bd67529c6ab,7400000,NaN,software,2000.0,Headquarters,"2030 First Avenue, Ste 300",,98121,Seattle,WA,USA,47.611331,-122.343058,M,7
18720,PubliAtis,52cdef7f4bab8bd67529c6a9,1000000,10.0,software,2006.0,World Headquarters,36 rue Vivienne,,75002,PARIS,None,FRA,48.869995,2.340695,M,1
18721,ANTs Software,52cdef7f4bab8bd67529c6ad,28300000,3.0,software,1979.0,Corporate Address,4514 Chamblee Dunwoody Rd.,Suite 188,30338,Dunwoody,GA,USA,37.590056,-122.347654,M,2
18733,Tiny Speck,52cdef7f4bab8bd67529c6b6,17200000,45.0,games_video,2009.0,,163 Hastings St W,,BC V6B 1H5,Vancouver,None,CAN,49.282455,-123.109217,M,1


In [20]:
country=["USA"]
usa3= usa3[(usa3["country_code"].isin(country))]

In [21]:
limit=500000
usa3 = usa3[usa3['total_money_raised']>=limit]

In [22]:
usa3

,name,_id,total_money_raised,number_of_employees,category_code,founded_year,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,Last_Digit,First_Digit
3,Wetpaint,52cdef7c4bab8bd675297d8a,39800000,47.0,web,2005.0,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253,M,3
3,Wetpaint,52cdef7c4bab8bd675297d8a,39800000,47.0,web,2005.0,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431,M,3
6,Geni,52cdef7c4bab8bd675297d91,16500000,18.0,web,2006.0,Headquarters,9229 W. Sunset Blvd.,,90069,West Hollywood,CA,USA,34.090368,-118.393064,M,1
9,StumbleUpon,52cdef7c4bab8bd675297d95,18500000,NaN,web,2002.0,,,,,San Francisco,CA,USA,37.775196,-122.419204,M,1
10,Gizmoz,52cdef7c4bab8bd675297d96,18100000,NaN,web,2003.0,None,None,None,None,Menlo Park,CA,USA,37.484130,-122.169472,M,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18557,Ponte Solutions,52cdef7f4bab8bd67529c604,7500000,NaN,software,2001.0,HQ,2570 West El Camino Real,Suite 250,94040,Mountain View,CA,USA,37.400945,-122.111318,M,7
18565,PaymentOne,52cdef7f4bab8bd67529c610,7000000,NaN,ecommerce,2000.0,,5883 Rue Ferrari,,95138,San Jose,CA,USA,37.248043,-121.776935,M,7
18658,Arkeia Software,52cdef7f4bab8bd67529c66c,4080000,50.0,software,1996.0,Corporate Headquarters,1808 Aston Ave,,92008,Carlsbad,CA,USA,33.132567,-117.292746,M,4
18719,Vigilos,52cdef7f4bab8bd67529c6ab,7400000,NaN,software,2000.0,Headquarters,"2030 First Avenue, Ste 300",,98121,Seattle,WA,USA,47.611331,-122.343058,M,7


In [37]:
usa_clean= usa3[["name","_id","total_money_raised","category_code","zip_code","latitude","longitude"]]


In [49]:
import numpy as np

def transformToGeoPoint(s):
    if np.isnan(s.latitude) or np.isnan(s.longitude):
        return None
    return {
        "type":"Point",
        "coordinates":[s.longitude, s.latitude]
    }

In [50]:
usa_clean
usa_clean["geopoint"] = usa_clean.apply(transformToGeoPoint, axis=1)

<ipython-input-50-72b6651a1dbf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usa_clean["geopoint"] = usa_clean.apply(transformToGeoPoint, axis=1)


In [52]:
usa_clean

,name,_id,total_money_raised,category_code,zip_code,latitude,longitude,geopoint
3,Wetpaint,52cdef7c4bab8bd675297d8a,39800000,web,98104,47.603122,-122.333253,"{'type': 'Point', 'coordinates': [-122.333253,..."
3,Wetpaint,52cdef7c4bab8bd675297d8a,39800000,web,10012,40.723731,-73.996431,"{'type': 'Point', 'coordinates': [-73.9964312,..."
6,Geni,52cdef7c4bab8bd675297d91,16500000,web,90069,34.090368,-118.393064,"{'type': 'Point', 'coordinates': [-118.393064,..."
9,StumbleUpon,52cdef7c4bab8bd675297d95,18500000,web,,37.775196,-122.419204,"{'type': 'Point', 'coordinates': [-122.419204,..."
10,Gizmoz,52cdef7c4bab8bd675297d96,18100000,web,None,37.484130,-122.169472,"{'type': 'Point', 'coordinates': [-122.169472,..."
...,...,...,...,...,...,...,...,...
18557,Ponte Solutions,52cdef7f4bab8bd67529c604,7500000,software,94040,37.400945,-122.111318,"{'type': 'Point', 'coordinates': [-122.111318,..."
18565,PaymentOne,52cdef7f4bab8bd67529c610,7000000,ecommerce,95138,37.248043,-121.776935,"{'type': 'Point', 'coordinates': [-121.776935,..."
18658,Arkeia Software,52cdef7f4bab8bd67529c66c,4080000,software,92008,33.132567,-117.292746,"{'type': 'Point', 'coordinates': [-117.292746,..."
18719,Vigilos,52cdef7f4bab8bd67529c6ab,7400000,software,98121,47.611331,-122.343058,"{'type': 'Point', 'coordinates': [-122.3430581..."


In [53]:
clean_offices["_id"] = clean_offices["_id"].apply(lambda e: str(e))
clean_offices.to_json("../input/clean_offices.json",orient="records")